In [26]:
import requests
import json
import pandas as pd
from config2 import api_key
from config2 import gkey
import csv
import gmaps

In [2]:
gmaps.configure(api_key=gkey)

In [3]:
beer_csv = pd.read_csv("addresses2.csv", delimiter=",")
beer_csv_df=pd.DataFrame(beer_csv)
beer_csv_df.head(1)

,Name,Address,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,City
0,10 Barrel Brewing Co.,1501 E St,San Diego,CA 92101,NaN,NaN,San Diego


In [6]:
top_5_csv = pd.read_csv("top_5.csv", delimiter=",")
top_5_csv_df=pd.DataFrame(top_5_csv)
top_5_csv_df = top_5_csv_df.drop(columns="Unnamed: 0")
top_5_csv_df.head()

,brewery,medal count
0,Pizza Port Carlsbad,24
1,AleSmith Brewing Co.,10
2,The Lost Abbey,9
3,Pizza Port Solana Beach,8
4,Pizza Port Ocean Beach,7


In [7]:
top_5 = list(zip(top_5_csv_df["brewery"], top_5_csv_df["medal count"]))
top_5

[('Pizza Port Carlsbad', 24),
 ('AleSmith Brewing Co.', 10),
 ('The Lost Abbey', 9),
 ('Pizza Port Solana Beach', 8),
 ('Pizza Port Ocean Beach', 7)]

In [8]:
biz_name = beer_csv_df["Name"]
addr = beer_csv_df["Address"]
city = beer_csv_df["City"]
addy = addr.values.tolist()
biz = biz_name.values.tolist()
cities = city.values.tolist()
len(cities)

152

In [5]:
# for index, row in beer_csv_df.iterrows():
#     biz, addy = row['Name'], row['Address']
#     print(a)

In [18]:
ids = []
name = []
address = []
lat = []
lng = []
rating = []
review = []
image = []
phone = []
x = -1
addy 
biz

skippedb = []
skippeda = []
for b, a, c in zip(biz, addy, cities):
    x += 1
    print(f"printing {b} || {x} of {len(biz)} ")
# for b, a, in beer_csv_df.iterrows():
    headers = {'Authorization': 'Bearer %s' % api_key}
    params = {'name':b,'address1':a,'city':c, 'state':'Ca', 'country':'US'}
    url = "https://api.yelp.com/v3/businesses/matches"
    try:
        response = requests.get(url, headers=headers, params=params)
        response_json = response.json()
        response_json
        idz = response_json["businesses"][0]["id"]
        addresses = response_json["businesses"][0]["location"]["address1"]
        lats = response_json["businesses"][0]["coordinates"]["latitude"]
        lngs = response_json["businesses"][0]["coordinates"]["longitude"]
        names = response_json["businesses"][0]["name"]
        name.append(names)
        address.append(addresses)
        lat.append(lats)
        lng.append(lngs)
        ids.append(idz)
    except:
        skippedb.append(b)
        skippeda.append(a)
        print('Skipping:', b)
        continue

for b, a, c, i, in zip(biz, addy, cities, ids):
    params = {'name':b,'address1':a,'city':c, 'state':'Ca', 'country':'US'}
    url = "https://api.yelp.com/v3/businesses/"
    response = requests.get(url + i, headers=headers, params=params)
    response_json = response.json()
    ratings = response_json["rating"]
    reviews = response_json["review_count"]
    images = response_json["image_url"]
    phones = response_json["display_phone"]
    image.append(images)
    phone.append(phones)
    review.append(reviews)
    rating.append(ratings)
        

printing 10 Barrel Brewing Co. || 0 of 152 
printing 2Kids Brewing Co. || 1 of 152 
printing 13 Point Brewing Company || 2 of 152 
printing 32 North Brewing Co. || 3 of 152 
printing Abnormal Beer Company || 4 of 152 
printing AleSmith Brewing Company || 5 of 152 
printing Align Brewing Co. || 6 of 152 
printing Alpine Beer Company || 7 of 152 
printing Amplified Ale Works || 8 of 152 
printing Amplified Ale Works || 9 of 152 
printing Arcana Brewing Co. || 10 of 152 
printing Attitude Brewing || 11 of 152 
printing Automatic Brewing Company || 12 of 152 
printing Aztec Brewing Company || 13 of 152 
printing Back Street Brewery || 14 of 152 
printing Bagby Beer Comapny || 15 of 152 
printing Ballast Point Brewing & Spirits - Home Brew Mart || 16 of 152 
printing Ballast Point Brewing & Spirits - Little Italy || 17 of 152 
printing Ballast Point Brewing & Spirits - Miramar || 18 of 152 
printing Ballast Point Brewing & Spirits - Scripps Ranch || 19 of 152 
printing Barn Brewery || 20 of

In [21]:
len(phone)

145

In [23]:
yelp_brew_dict = {"Name":name, "Review Count":review, "Rating":rating}
yelp_brew_df = pd.DataFrame(yelp_brew_dict)
yelp_brew_df["Lat"] = lat
yelp_brew_df["lng"] = lng
yelp_brew_df["Phone#"] = phone
yelp_brew_df["URL"] = images
yelp_brew_df.head(2)

,Name,Review Count,Rating,Lat,lng,Phone#,URL
0,10 Barrel Brewing - San Diego,364,4.5,32.714559,-117.150229,(619) 578-2311,https://s3-media2.fl.yelpcdn.com/bphoto/6S8tUq...
1,2kids Brewing Co,86,4.0,32.896564,-117.137428,(858) 480-5437,https://s3-media2.fl.yelpcdn.com/bphoto/6S8tUq...


In [25]:
lat_list = yelp_brew_df["Lat"].tolist()
lng_list = yelp_brew_df["lng"].tolist()
rat_list = yelp_brew_df["Rating"].astype(float)
coordinates = list(zip(lat_list, lng_list))
locations = yelp_brew_df[["Lat", "lng"]].astype(float)
brew_name = yelp_brew_df["Name"].tolist()
review_ct = yelp_brew_df["Review Count"].tolist()
phone_lst = yelp_brew_df["Phone#"].tolist()
url_lst = yelp_brew_df["URL"].tolist()

In [34]:
locations
figure_layout = {'width': '900px',
                 'height': '500px',
                 'border': '1px solid black',
                 'padding': '1px',
                 'margin': '0 auto 0 auto',
                 'Bold':True
                }
fig = gmaps.figure(map_type='HYBRID')
markers = gmaps.marker_layer(coordinates, 
                             hover_text=[f"Name: {brew} \n Rating: {rating}\n Reviews: {rev}\n Phone: {ph}" for brew, rating, rev, ph, url in zip(brew_name, rat_list, review_ct, phone_lst, url_lst)], 
                             label='', 
                             info_box_content=[f"Brewery Rating: {rating}" for rating in rat_list], 
                             display_info_box=None
                            )
# top_5_layer = gmaps.symbol_layer(
#     starbucks_df, fill_color="red",
#     stroke_color="red", scale=5
fig.add_layer(markers)
#_____________________________

fig
#__________________________________

Figure(layout=FigureLayout(height='420px'))

In [64]:
# brewery_layer = gmaps.symbol_layer(
#     locations, fill_color='red',
#     stroke_color='red', scale=2,
#     info_box_content=[f"Brewery Name: {brewery}" for brewery in brew_name]
# )


# fig = gmaps.figure()
# fig.add_layer(brewery_layer)
# fig

In [16]:
#for b, a, c in zip(biz, addy, cities):
    #print(f"printing {b} || {x} of {len(biz)} ")
# for b, a, in beer_csv_df.iterrows():
    

headers = {'Authorization': 'Bearer %s' % api_key}
params = {'name':"10 Barrel Brewing Co",'address1':"501 E St",'city':"San Diego", 'state':'Ca', 'country':'US'}
url = "https://api.yelp.com/v3/businesses/"
response = requests.get(url + "8nEOrZJXEPbB7omJwyM1JQ", headers=headers, params=params)
response_json = response.json()
response_json

{'id': '8nEOrZJXEPbB7omJwyM1JQ',
 'alias': '10-barrel-brewing-san-diego-san-diego',
 'name': '10 Barrel Brewing - San Diego',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/sSsglk_cRmSELCL-FhMWHQ/o.jpg',
 'is_claimed': True,
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/10-barrel-brewing-san-diego-san-diego?adjust_creative=YMS3nsDc5AUcix9rnsV4oQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=YMS3nsDc5AUcix9rnsV4oQ',
 'phone': '+16195782311',
 'display_phone': '(619) 578-2311',
 'review_count': 364,
 'categories': [{'alias': 'breweries', 'title': 'Breweries'},
  {'alias': 'pubs', 'title': 'Pubs'},
  {'alias': 'tradamerican', 'title': 'American (Traditional)'}],
 'rating': 4.5,
 'location': {'address1': '1501 E St',
  'address2': '',
  'address3': None,
  'city': 'San Diego',
  'zip_code': '92101',
  'country': 'US',
  'state': 'CA',
  'display_address': ['1501 E St', 'San Diego, CA 92101'],
  'cross_streets': '16th St & 15th St'},
 'coordinates': 

In [ ]:
8nEOrZJXEPbB7omJwyM1JQ

In [33]:
yelp_brew_df.head()

,Name,Review Count,Rating,Lat,lng,Phone#,URL
0,10 Barrel Brewing - San Diego,364,4.5,32.714559,-117.150229,(619) 578-2311,https://s3-media2.fl.yelpcdn.com/bphoto/6S8tUq...
1,2kids Brewing Co,86,4.0,32.896564,-117.137428,(858) 480-5437,https://s3-media2.fl.yelpcdn.com/bphoto/6S8tUq...
2,13 Point Brewing Company,34,4.5,32.742786,-117.024910,,https://s3-media2.fl.yelpcdn.com/bphoto/6S8tUq...
3,32 North Brewing Co,170,4.5,32.882744,-117.164986,(619) 363-2622,https://s3-media2.fl.yelpcdn.com/bphoto/6S8tUq...
4,Abnormal Beer Company,6,5.0,33.023850,-117.085750,(858) 618-2463,https://s3-media2.fl.yelpcdn.com/bphoto/6S8tUq...


In [126]:
for i, row in yelp_brew_df.iterrows():
    name = row['Name']
    Lat = row['Lat']

10 Barrel Brewing - San Diego
2kids Brewing Co
13 Point Brewing Company
32 North Brewing Co
Abnormal Beer Company
AleSmith Brewing Company
Align Brewing Company
Alpine Beer Company - Tasting Room
Amplified Ale Works
Amplified Ale Works
Arcana Brewing Company
Attitude Brewing Company
Automatic Brewing Company
Aztec Brewery
Lamppost Pizza - Backstreet Brewery
Bagby Beer Company
Home Brew Mart - Ballast Point
Ballast Point Little Italy
Ballast Point Miramar
Ballast Point Scripps Ranch
Barn Brewery
Barrel Harbor Brewing
BattleMage Brewing Company
Bay Bridge Brewing
Bay City Brewing Company
Beach Grease Beer Company
Bear Roots Brewing
Belching Beaver Pub980
Belching Beaver Brewery - Oceanside
Belching Beaver Brewery Tavern & Grill
Benchmark Brewing Co
Bitter Brothers Brewing
BLACK PLAGUE Brewing
BNS Brewing & Distilling
Bolt Brewery
Booze Brothers Brewing
Border X Brewing
Breakwater Brewing Company
Burgeon Beer Co
Burning Beard Brewing Company
California Wild Ales
Chula Vista Brewery
Circle